In [61]:
from helpers import generate_random_payments, convert_to_df, process_ingress_payments
from tokenomics import ERC20Token, Bucket

In [62]:
#build contracts set
token = ERC20Token()
bkt_four = Bucket(token=token, name="bkt_04")
bkt_three = Bucket(token=token, max_volume=300000, overflow_bkt=bkt_four, name="bkt_03")
bkt_two = Bucket(token=token, max_volume=200000, overflow_bkt=bkt_three, name="bkt_02")
bkt_one = Bucket(token=token, max_volume=100000, overflow_bkt=bkt_two, name="bkt_01")

In [63]:
# generate random payments
payments = generate_random_payments()
# process each payment and calculate how buckets get filled
processed = process_ingress_payments(payments, token, bkt_one, bkt_two, bkt_three, bkt_four)
columns = processed[0]
dataframe = convert_to_df(processed)

In [64]:
# display the first records of df
dataframe.head()

,income,bkt_01,bkt_02,bkt_03,bkt_04
date,,,,,
2019-01-02 23:47:26,781,781,0,0,0
2019-01-05 05:19:17,5893,6674,0,0,0
2019-01-07 10:19:26,772,7446,0,0,0
2019-01-07 12:32:07,8816,16262,0,0,0
2019-01-09 04:38:25,3219,19481,0,0,0


In [83]:
# aggregation pattern declares which column should be summarized 
# and which should have latest value after
aggpattern = { columns[1] : 'sum' }
for k in columns[2:]:
    aggpattern[k] = 'last'
    
# Aggregate by year+month
grouped_and_aggregated_df = dataframe.groupby(lambda x : x.strftime('%y-%m'), sort=False).agg(aggpattern)
grouped_and_aggregated_df
# Add accumulated summary for investments
grouped_and_aggregated_df['cum_sum'] = grouped_and_aggregated_df.income.cumsum()
grouped_and_aggregated_df

,income,bkt_01,bkt_02,bkt_03,bkt_04,cum_sum
19-01,116022,100000,16022,0,0,116022
19-02,39376,100000,55398,0,0,155398
19-03,56826,100000,112224,0,0,212224
19-04,75347,100000,187571,0,0,287571
19-05,66764,100000,200000,54335,0,354335
19-06,81741,100000,200000,136076,0,436076
19-07,67801,100000,200000,203877,0,503877
19-08,114109,100000,200000,300000,17986,617986
19-09,107138,100000,200000,300000,125124,725124
19-10,75310,100000,200000,300000,200434,800434


In [81]:
import pandas as pd
import numpy as np
from random import randint
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

traces = []
for c in columns[1:]:
    trace = {
        'x': grouped_and_aggregated_df.index,
        'y': grouped_and_aggregated_df[c],
        'name': c,
    'type': 'bar',
    'yaxis': 'y'
    }
    traces.append(trace)

trace_cumsum = {
    'x': grouped_and_aggregated_df.index, 
    'y': grouped_and_aggregated_df.cum_sum,
    'name': 'Total',
    'type': 'scatter',
    'yaxis': 'y2',
    'mode':'markers+lines',
    'line': {
        'shape': 'hvh',
    }
}
traces.append(trace_cumsum)

layout = {
    'xaxis': {
        'title': 'X axis'
    },
    'yaxis': {
        'title': 'Y Buckets',
        'side': 'left',
        'zeroline': True
    },
    'yaxis2': {
        'title': 'Y Total invest',
        'side': 'right',
        'overlaying': 'y',
    },
    'barmode': 'group',
    'title': 'Investments and buckets filling', 
}

iplot({'data': traces, 'layout': layout})